In [20]:
using RegressionDynamicCausalModeling, MAT, Plots, HDF5
using FilePathsBase, Serialization, Random, UnPack, Distributed

In [2]:
#addprocs(3; exeflags="--project")

3-element Vector{Int64}:
 2
 3
 4

In [3]:
function h5_save_output(output_srcdm, output_rdcm, filename::String)
    h5open(filename, "w") do file
        create_group(file, "srdcm")
        create_group(file, "rdcm")
        write(file, "srdcm/m_all", output_srdcm.m_all)
        write(file, "srdcm/F_r", output_srdcm.F_r)
        write(file, "srdcm/z_all", output_srdcm.z_all)
        write(file, "rdcm/m_all", output_rdcm.m_all)
        write(file, "rdcm/F_r", output_rdcm.F_r)
        write(file, "rdcm/z_all", output_rdcm.z_all)
    end
end

h5_save_output (generic function with 1 method)

In [4]:
function save_ser_matrix(matrix, filename::String)
    open(filename, "w") do file
        serialize(file, matrix)
    end
end

function load_ser_matrix(filename::String)
    open(filename, "r") do file
        return deserialize(file)
    end
end

load_ser_matrix (generic function with 1 method)

In [12]:
function process_to_rdcm(dataDir::String, sub::String, targetDir::String, p0::Float64)
    dcm = load_DCM(joinpath(dataDir, sub, "DCM.mat"))
    opt_srdcm = Options(SparseInversionParams(; reruns=50, restrictInputs=true); synthetic=false, rng=MersenneTwister(42))
    opt_rdcm = Options(RigidInversionParams(); synthetic=false, rng=MersenneTwister(42))

    rdcm = RigidRdcm(dcm) # convert the DCM to a rDCM model
    dcm_test = LinearDCM(dcm.a,rdcm.c,dcm.scans,dcm.nr,InputU(rdcm.U.u, rdcm.U.dt),dcm.Y,dcm.Ep)
    srdcm = SparseRdcm(dcm_test; inform_p0=true, p0=p0)
    
    # Estimate effective connectivity
    output_srdcm = invert(srdcm, opt_srdcm)
    output_rdcm = invert(rdcm, opt_rdcm)

    p0_out = string(p0*100)

    # save matrix
    h5_save_output(output_srdcm, output_rdcm, "$targetDir/$(sub)_inv_mF_$(p0_out).h5")
    save_ser_matrix(output_srdcm.m_all, "$targetDir/$(sub)_srdcm_m_$(p0_out).mat")
    save_ser_matrix(output_rdcm.m_all, "$targetDir/$(sub)_rdcm_m_$(p0_out).mat")
    save_ser_matrix(output_srdcm.Σ_all, "$targetDir/$(sub)_srdcm_sigma_$(p0_out).mat")
    save_ser_matrix(output_rdcm.Σ_all, "$targetDir/$(sub)_rdcm_sigma_$(p0_out).mat")
    save_ser_matrix(output_srdcm.F_r, "$targetDir/$(sub)_srdcm_F_$(p0_out).mat")
    save_ser_matrix(output_rdcm.F_r, "$targetDir/$(sub)_rdcm_F_$(p0_out).mat")

end

process_to_rdcm (generic function with 1 method)

In [18]:
function main()
    #dataDir = "/cluster/scratch/spruthi/project4/SRPBS_OPEN/data"
    dataDir = "D:\\zurich_spring24\\tn\\project4\\SRPBS_OPEN\\data"
    sparsity_p0 = 0.20

    #for file in readdir(dataDir)
    for file in ["sub-1363", "sub-1410"]
        #if file == "Brainnetome2016" || file == "sub-1367" || file == "sub-1378" || file == "sub-1380" || file == "sub-1385" || file == "sub-1392" || file == "sub-1402" || file == "sub-1404" || file == "sub-1406" || file == "sub-1407"
        #    continue
        #end

        dataFile = joinpath(dataDir, file)
        targetDir = joinpath(dataFile, "sparseInversion")

        if !ispath(targetDir)
            mkdir(targetDir)
            println("$targetDir created")
        end

        #println("$target_file does not exist yet")
        
        try
            process_to_rdcm(dataDir, file, targetDir, sparsity_p0)
        catch e
            println("An error occurred: $(typeof(e)), err: $(e)")
            continue
        end
    end
end

main (generic function with 1 method)

In [ ]:
main()

Running model inversion (sparse rDCM)


[ Info: No input found-> constructing resting-state DCM.
[ Info: Ep field missing, using default values.
[ Info: Found linear DCM.
[ Info: No input found-> constructing resting-state DCM.


An error occurred: InterruptException, err: InterruptException()
Running model inversion (sparse rDCM)

[ Info: Ep field missing, using default values.
[ Info: Found linear DCM.


Progress:   1%|█                                        |  ETA: 1:13:24

An error occurred: InterruptException, err: InterruptException()


InterruptException: InterruptException:

In [17]:
ispath("D:\\zurich_spring24\\tn\\project4\\SRPBS_OPEN\\data\\sub-1363\\sparseInversion")

true

In [6]:
p0 = 0.254
p0_check = string(floor(Int, p0*100))

"25"